<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9回目の提出

ここまでのtry
*   Light GBM、特徴量12個くらい。MinMaxScalerで正規化
*   基準を0.1以上をOKとしたらクリアになった。
*   これは完全に過学習を起こしている。
*　 crossvalidationがちゃんと働いてないな。
*　 cataegorical featuresという形でもうできることがわかったぞ


---


今後の課題
*   optunaを入れて調整しよう
*   過学習の判断をできるようにしよう
*   Minmaxとかの正規化ってcategoricaldataにもやっていいの？
*   
*   


In [1]:
#importまとめ。pandas-profilingは少々導入が面倒
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file
%matplotlib inline

warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-h3wbbawq
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-h3wbbawq
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=dbd3e85763c2e9593b57283bee91bf40eea64dc001f4ce18ed889658732c14aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-0l5v3787/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
Successfully built pandas-profiling


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#train,test,submit_sampleのみっつがそろっているか確認
ls

oof_train_kfold.csv  submission_lightgbm_kfold.csv  train.csv
sample_data/         submit_sample.csv
submission.csv       test.csv


In [3]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

In [4]:
ls

oof_train_kfold.csv  submission_lightgbm_kfold.csv  train.csv
sample_data/         submit_sample.csv
submission.csv       test.csv


In [35]:
#上手く列指定ができなかったので、先頭にheaderをつける
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
#test,train,その足し合わせdataの三種を作る
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["job"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["marital"].replace(['married','single','divorced'],[2,1,0],inplace=True)
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,2,1,0],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,2,1,0],inplace=True)
data["job"] = data["job"].astype(np.int64)

#全体のうち、消したいモノだけ選ぶ
all_columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome']
delete_columns = ['id','default','day','month','pdays','balance''duration']
remain_columns = list(set(all_columns)-set(delete_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]


**------------------cross validation--------------------**


**------light GBM-------**

In [36]:
#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。
y_train = train['y']
X_train = train.drop('y', axis = 1)
X_test = test.drop('y', axis = 1)

# MinmaxScalarを使って正規化。
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
# 訓練用のデータを正規化する
X_train_norm = mmsc.fit_transform(X_train)
# 訓練用データを基準にテストデータも正規化
X_test_norm = mmsc.transform(X_test)
X_train=X_train_norm
X_test=X_test_norm
#正規化するとarrayになっていたので、DataFrameに戻す
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
#headerが抜けているので付け直す
X_train.columns=remain_columns
X_test.columns=remain_columns

In [45]:
#欲しいデータが見えているか確認
data

,age,job,marital,education,balance,housing,loan,contact,duration,campaign,previous,poutcome,y
0,31,4,2,3,12294,1,0,2,101,3,0,0,0.0
1,29,7,1,2,43027,0,0,2,158,2,0,1,1.0
2,35,1,2,2,12252,1,0,2,351,1,0,2,0.0
3,31,2,2,3,99121,1,1,0,658,2,0,2,0.0
4,48,10,2,1,42005,1,0,1,177,1,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,2,2,98357,1,0,2,101,2,0,2,NaN
18046,34,0,2,3,29621,1,0,2,345,1,0,1,NaN
18047,34,3,1,3,94260,1,0,0,121,2,0,1,NaN
18048,31,2,1,3,65483,1,0,0,345,2,0,1,NaN


In [37]:
#LightGBMに入れる。
from sklearn.model_selection import KFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

#categorical dataの入れ忘れに注意
categorical_features = ["job","marital","education","housing","loan","contact","poutcome"]

params = {
    'objective': 'binary',
    'max_bin': 400,
    'learning_rate': 0.01,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)


In [39]:
    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.259381	valid_1's binary_logloss: 0.27128
[20]	training's binary_logloss: 0.251601	valid_1's binary_logloss: 0.262857
[30]	training's binary_logloss: 0.245845	valid_1's binary_logloss: 0.256638
[40]	training's binary_logloss: 0.241302	valid_1's binary_logloss: 0.251787
[50]	training's binary_logloss: 0.23754	valid_1's binary_logloss: 0.247852
[60]	training's binary_logloss: 0.234281	valid_1's binary_logloss: 0.244449
[70]	training's binary_logloss: 0.231438	valid_1's binary_logloss: 0.241502
[80]	training's binary_logloss: 0.228962	valid_1's binary_logloss: 0.239022
[90]	training's binary_logloss: 0.226786	valid_1's binary_logloss: 0.23681
[100]	training's binary_logloss: 0.224854	valid_1's binary_logloss: 0.234876
[110]	training's binary_logloss: 0.223199	valid_1's binary_logloss: 0.233342
[120]	training's binary_logloss: 0.221725	valid_1's binary_logloss: 0.232057
[130]	training's binary_lo

In [40]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.2231732415940769]
0.2231732415940769


In [41]:
from sklearn.metrics import accuracy_score
percentage=0.05

y_pred_oof = (oof_train > percentage).astype(int)
accuracy_score(y_train, y_pred_oof)

0.8539852398523985

In [42]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > percentage).astype(int)


sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(8656, 18050)

In [43]:
sub

,A,B
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1
...,...,...
18045,18045,1
18046,18046,0
18047,18047,0
18048,18048,0


-----ここまでcross validation-------

In [44]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>